## Benchmarking task - Summarization

#### Install python packages requested by benchmarking

If you have not install the requested python libraries, uncomment the following command to run the installation.

In [ ]:
#!pip install -r ../peccyben/requirements.txt

#### Import libraries

In [ ]:
import pandas as pd
import sys

sys.path.insert(0, '../')

from peccyben.summarizationtask import Summ_Ben
from peccyben.utils import Ben_Save
from peccyben.promptcatalog import Prompt_Template_Gen

#### Configuration

Setup your environment parameters 

* **BENCH_KEY**: a unique keyname for your benchmarking in this round 
* **S3_BUCKET**: the S3 buckt you created for the benchmarking    
* **TASK_FOLDER**: the task folder you created under the S3 bucket   
* **INPUT_FILE**: the file name of the dataset you prepared for benchmarking    
* **METRICS_LIST**: the metrics we provide for the text summarization task      
* **BEDROCK_REGION**: the AWS region that the model benchmarking runs on Bedrock
* **COST_FILE**: the price file used for calculating model inference cost 

In [ ]:
BENCH_KEY = 'PeccyBen_202503_xsum'
S3_BUCKET = 'genai-sdo-llm-ben-20240310'
TASK_FOLDER = 'ben-summ'
INPUT_FILE = 'xsum_100.csv'
METRICS_LIST = ['Inference_Time','Input_Token','Output_Token','Throughput','RougeL-sum','Semantic_Similarity','BERT-F1','LJ_Score','Toxicity','Cost','Cache_Input_Token','Cache_Output_Token']
BEDROCK_REGION = 'us-east-1'
COST_FILE = 'bedrock_od_public.csv'

Results_summ = pd.DataFrame()
Results_summ = Results_summ.assign(metric_name=METRICS_LIST) 
Results_summ = Results_summ.set_index('metric_name')

#### Task specific setting

* Configure your **prompt** in the prompt catalog (prompt_catalog.json), and configure the prompt_catalog_id
* Set the **LLM hyperparameter** in model_kwargs. For the models on Bedrock, refer to [inferenceConfig](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference-call.html)
* Set two **LLM-judge models** (judge_model_1, judge_model_2) for the LLM-judge for the summarization task

In [ ]:
prompt_catalog_id = "summ-1"

model_kwargs = {
    'maxTokens': 512, 
    'topP': 0.9, 
    'temperature': 0
}   

judge_model_1 = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
judge_model_2 = "us.deepseek.r1-v1:0"

#### Specify the model and other settings for benchmarking

Invoke **Summ_Ben** function to conduct the benchmarking for one selected model, repeat for multiple models you want to benchmark

* **method**: set "Bedrock" for the models on Bedrock
* **region**: configured in the previous step
* **model_id**: specify the Model ID for the model endpoint
* **jm1**: configured in previous step
* **jm2**: configured in previous step
* **model_kwargs**: configured in previous step
* **prompt_template**: prompt template based on the prompt configured in previous step
* **s3_bucket**: configured in previous step
* **file_name**: configured in previous step
* **BENCH_KEY**: configured in previous step
* **task_folder**: configured in previous step
* **cost_key**: set "public" when using AWS public pricing to calculate the cost
* **save_id**: the model name displayed in the report 
* **SLEEP_SEC**: you can configure "sleep and retry" when throtting, for example, set SLEEP_SEC = 10 to wait for 10 seconds between each inference
* **SAMPLE_LEN**: you can configure the number of samples for inference
* **PP_TIME**: if you want to run model inference for multiple rounds, set the number of rounds here.  
* **cacheconf**: set "default" to enable Bedrock Prompt Caching in the inference, "None" to disable
* **latencyOpt**: set "optimized" to enable Bedrock Latency Optimized Inference, "None" to disable

In [ ]:
# Haiku 3.5
model_id = 'us.anthropic.claude-3-5-haiku-20241022-v1:0' 
save_id = 'haiku-3.5'

prompt_template = Prompt_Template_Gen(model_id, prompt_catalog_id)

Results_summ[save_id] = Summ_Ben(method="Bedrock",
                                 region=BEDROCK_REGION,
                                 model_id=model_id,
                                 jm1=judge_model_1,
                                 jm2=judge_model_2,
                                 model_kwargs=model_kwargs,
                                 prompt_template=prompt_template,
                                 s3_bucket=S3_BUCKET,
                                 file_name=INPUT_FILE,
                                 BENCH_KEY=BENCH_KEY,
                                 task_folder=TASK_FOLDER,
                                 cost_key=COST_FILE,
                                 save_id=save_id,
                                 SLEEP_SEC=20,SAMPLE_LEN=2,
                                 PP_TIME=2,
                                 cacheconf="None",latencyOpt="None")

Results_summ

In [ ]:
# nova-micro
model_id = 'us.amazon.nova-micro-v1:0' 
save_id = 'nova-micro'

prompt_template = Prompt_Template_Gen(model_id, prompt_catalog_id)
#print(prompt_template)

Results_summ[save_id] = Summ_Ben(method="Bedrock",
                                 region=BEDROCK_REGION,
                                 model_id=model_id,
                                 jm1=judge_model_1,
                                 jm2=judge_model_2,
                                 model_kwargs=model_kwargs,
                                 prompt_template=prompt_template,
                                 s3_bucket=S3_BUCKET,
                                 file_name=INPUT_FILE,
                                 BENCH_KEY=BENCH_KEY,
                                 task_folder=TASK_FOLDER,
                                 cost_key=COST_FILE,
                                 save_id=save_id,
                                 SLEEP_SEC=20,SAMPLE_LEN=2,
                                 PP_TIME=2,
                                 cacheconf="None",latencyOpt="None")

Results_summ

#### Generate benchmarking report

Invoke **Ben_Save** function to generate your benchmarking report and cost-performance analysis, all the results are stored in S3 bucket

* **Results_summ**: the benchmarking results generated in previous step
* **S3_BUCKET**: configured in previous step
* **BENCH_KEY**: configured in previous step
* **TASK_FOLDER**: configured in previous step
* **perf_metric**: select the performance metric from the metrics list in the previous step, for cost-performance analysis, for example, to analyze the accuracy by LLM-judge score with cost, set "LJ_Score".  
* **top_x**: set the top x number of models you want to run the cost-performance analysis 
* **TITLE**: specify a title for the reports and charts


In [ ]:
perf_metric = 'LJ_Score'
top_x = 5

Ben_Save(Results_summ,S3_BUCKET,BENCH_KEY,TASK_FOLDER,perf_metric,top_x,TITLE="Summarization-Task")